### Prompt

In [1]:
import bs4, tiktoken, numpy as np, os

from langchain import hub
from langchain.load import dumps, loads
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from operator import itemgetter

from dotenv import load_dotenv
load_dotenv()

learn_api_key = os.environ['LANGCHAIN_API_KEY']
openai_api_key = os.environ['OPENAI_API_KEY']

In [2]:
# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [3]:
generate_queries = (
    prompt_rag_fusion
    | ChatGroq(model='sentence-transformers/all-MiniLM-L6-v2', temperature=0)
    | StrOutputParser()
    | (lambda x: x.split('\n'))
)

In [4]:
def reciprocal_rank_fusion(results: list[list], k=60):
    """Reciprocal_rank_fusion that takes multiple lists of ranked documents
    and an optional parameter k used in the RRF formula """

    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)
    
    # Sort the documents based on their fused scores in decreasing order to get the final reranked results
    reranked_results = [
        (loads(docs), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x:x[1], reverse=True)
    ]

    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question":question})
len(docs)

NameError: name 'retriever' is not defined

In [ ]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""
propmt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})